# Gemicai tutorial 5: ClassifierTree
In this tutorial everything concerning the ClassifierTree will be explained

In [1]:
import gemicai as gem
import torch
import torchvision.models as models

## 3.1 Initialising ClassifierTree
First, we have to initialize the tree, to do this we need the following parameters

## 3.2 Training the tree
To train the tree, first retrive it from its directory

In [2]:
dx_tree_path = '/mnt/SharedStor/trees/dx_tree'

tree = gem.ClassifierTree.from_dir(dx_tree_path)

print(tree)

|   Depth | Label            |   Classifiers |   Avg. classes |
|---------+------------------+---------------+----------------|
|       0 | BodyPartExamined |             1 |           21   |
|       1 | StudyDescription |            20 |            9.7 |


To train the whole tree at once, simply use the train function. We also need a dataset, to learn more about datasets check out tutorial 3

In [3]:

dataset_path = '/mnt/SharedStor/train_dataset/DX/'
relevant_labels = ['BodyPartExamined', 'StudyDescription']
dataset = gem.DicomoDataset.get_dicomo_dataset(dataset_path, labels=relevant_labels)

# tree.train(dataset, epochs=2)


Training a ClassifierTree like this is a good start, but to get the most out of your tree the indidual nodes will need tweaking. To inspect the accuracy, of the tree, and where it needs tweaking, call evaluate.

In [4]:

testset = gem.DicomoDataset.get_dicomo_dataset('/mnt/SharedStor/eval_dataset/DX/', labels=relevant_labels)

tree.evaluate(testset)


|  Node | Depth | Parents              |  Classes |  Test size |   Test acc | 
|-------+-------+----------------------+----------+------------+------------|
|     1 |     1 | .                    |       21 |       1200 |      3.08% | 
|     2 |     1 | FOOT                 |       14 |        151 |     17.22% | 
|     3 |     1 | PELVIS               |       14 |         60 |      1.67% | 
|     4 |     1 | SKULL                |        2 |          3 |       0.0% | 
|     5 |     1 | SHOULDER             |       17 |        102 |       9.8% | 
|     6 |     1 | TSPINE               |        3 |          9 |     44.44% | 
|     7 |     1 | CSPINE               |        2 |         23 |     30.43% | 
|     8 |     1 | CLAVICLE             |        5 |          1 |       0.0% | 
|     9 |     1 | SSPINE               |        1 |          0 |       N/A% | 
|    10 |     1 | ARM                  |        4 |          3 |       0.0% | 
|    11 |     1 | ANKLE                |       10 |  

From the evaluation we can see that node 2 appears to not be performing very well. To train this individual node, we first need to find it's path

In [5]:

# By default the filename of the node is <node.label>.gemnode
node_path = '/mnt/SharedStor/trees/dx_tree/FOOT/StudyDescription.gemnode'

node = gem.ClassifierNode.from_file(node_path)

# Select neural network from the node
net = node.classifier
print(type(net))


<class 'gemicai.Classifier.Classifier'>


As you can see, net is an instance of Gemicai.Classifier, we already know how to train a Classifier! For more information about Gemicai.Classifier see tutorial 2. 

In [8]:

trainset_path = '/mnt/SharedStor/train_dataset/DX/'
trainset = gem.DicomoDataset.get_dicomo_dataset(trainset_path, labels=relevant_labels)
trainsubset = trainset.subset({'BodyPartExamined': 'FOOT'})[1]
testsubset = testset.subset({'BodyPartExamined': 'FOOT'})[1]

trainsubset.summarize('StudyDescription')
testsubset.summarize('StudyDescription')


| Class (StudyDescription)   |   Frequency |
|----------------------------+-------------|
| Voet links                 |         164 |
| Enkel links                |          98 |
| Voet rechts                |         176 |
| Voet links kind            |           6 |
| Calcaneus rechts           |          14 |
| Voet beiderzijds           |          95 |
| Voet rechts kind           |           2 |
| Enkel rechts               |         121 |
| Teen rechts                |           9 |
| Teen links                 |           7 |
| Calcaneus links            |          13 |
| Enkel beiderzijds          |          13 |
| Teen links kind            |           2 |

Total number of training images: 720 
Total number of classes: 13

| Class (StudyDescription)   |   Frequency |
|----------------------------+-------------|
| Voet rechts                |          35 |
| Voet links                 |          45 |
| Enkel links                |          18 |
| Calcaneus links            |  

In [ ]:
net.train(trainsubset, test_dataset=testsubset, epochs=10, verbosity=2)

| Epoch | Avg. loss | Train Acc. | Test Acc.  | Elapsed  |   ETA    |
|-------+-----------+------------+------------+----------+----------|
|     1 | 0.3062769 | 18.47%     | 17.22%     | 00:00:08 | 03:58:07 |


In [13]:
net.evaluate(testsubset, verbosity=2)

| <class 'list'> | <class 'list'> | <class 'list'> |
| Class             |   Total |   Correct | Acc   |
|-------------------+---------+-----------+-------|
| Voet links        |      45 |         2 | 4.4%  |
| Enkel links       |      18 |         1 | 5.6%  |
| Voet rechts       |      35 |        23 | 65.7% |
| Voet links kind   |       0 |         0 | -     |
| Enkel rechts      |      23 |         0 | 0.0%  |
| Calcaneus rechts  |       6 |         0 | 0.0%  |
| Voet beiderzijds  |      16 |         0 | 0.0%  |
| Voet rechts kind  |       0 |         0 | -     |
| Teen rechts       |       0 |         0 | -     |
| Teen links        |       0 |         0 | -     |
| Calcaneus links   |       3 |         0 | 0.0%  |
| Enkel beiderzijds |       5 |         0 | 0.0%  |
| Teen rechts kind  |       0 |         0 | -     |
| Teen links kind   |       0 |         0 | -     | 



(17.22, 151, 26)

If you want, you can assign a whole net classifier to the node. First train the new and improved Classifier, then assign it to the node's classifier, then save the node.

In [14]:

resnet18 = models.resnet18(pretrained=False)

newnet = gem.Classifier(resnet18, trainset.classes('StudyDescription'), enable_cuda=True)


In [ ]:

newnet.train(trainsubset, test_dataset=testsubset, epochs=25)

newnet.evaluate(testsubset, verbosity=2)


In [ ]:

node.classifier = newnet

node.save()


Now when, evaluating the tree again, we can see that the accuracy of node NUMBER improved!

In [ ]:

tree.evaluate(testset)
